In [1]:
import gspread
from gspread_dataframe import get_as_dataframe
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import set_with_dataframe
import pyodbc
import pandas as pd
from datetime import datetime
import numpy as np

In [2]:
# Setup
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("python-459011-73acbdff4227.json", scope)
client = gspread.authorize(creds)

In [3]:
# Open Sheet
sheet = client.open_by_key("1Feh5lJStiEkMjT0qkLl8yu81Tbi1WjIeILcxrk13FsI") # Master Sheets ( Dimensions Sheet)

#### Extracting Masters from GoogleSheetBackend Masters

In [4]:
StoreMaster = sheet.worksheet("StoreMaster")
SOSTargetMaster = sheet.worksheet("SOSTargetMaster")
ProductMaster = sheet.worksheet("ProductMaster")
Overall_Master_Product = sheet.worksheet("Overall_Master_Product")

NPF_Master_SOS = sheet.worksheet("NPF_Master_SOS")


In [5]:
StoreMaster = get_as_dataframe(StoreMaster)
SOSTargetMaster = get_as_dataframe(SOSTargetMaster)
ProductMaster = get_as_dataframe(ProductMaster)

Overall_Master_Product = get_as_dataframe(Overall_Master_Product)
NPF_Master_SOS = get_as_dataframe(NPF_Master_SOS)


In [6]:
StoreMasterRevised = StoreMaster[
    ['LIS Store Code', 'Old DT Store Code', 'DT Store Code', 'Amazing ID', 'Emp User ID','LIS User Id','Employee Name']
]

StoreMasterRevised = StoreMasterRevised.rename(
    columns={
        'LIS Store Code': 'New_DT_Store_Code',
        'Old DT Store Code': 'OLD_DT_Store_Code',
        'DT Store Code': 'LIS_StoreCode',
        'Amazing ID': 'Old_Amazing_EMP_ID',
        'Emp User ID': 'New_Amazing_EMP_ID',
        'LIS User Id':'LISUserId',
        'Employee Name':'EmployeeName'
    }
)

In [7]:
## NEED TO HAVE CAMP ID ALWYAS IN UPPER CASE IN MASTER ANN COVERAGE AND SO ON

StoreMasterRevised['New_Amazing_EMP_ID'] = (
    StoreMasterRevised['New_Amazing_EMP_ID']
    .astype(str)
    .str.upper()
)

StoreMasterRevised['Old_Amazing_EMP_ID'] = (
    StoreMasterRevised['Old_Amazing_EMP_ID']
    .astype(str)
    .str.upper()
)

In [8]:
StoreMasterRevised.head()

New_DT_Store_Code OLD_DT_Store_Code  LIS_StoreCode Old_Amazing_EMP_ID  \
0       DT02DMA2001       DT02DMA2001  CLG-Store-497               X157   
1       DT02DMA2003       DT02DMA2003  CLG-Store-491          IN0028M57   
2       DT02DMA2005       DT02DMA2005  CLG-Store-494         IN0028M154   
3         DT02MET54       DT02MET54-1  CLG-Store-499         IN0028M475   
4       DT02RRLFRDF       DT02RRLFRDF  CLG-Store-496         IN0028M326   

  New_Amazing_EMP_ID    LISUserId    EmployeeName  
0            CAMP059  CLG-ISP-507    Mukesh Kumar  
1            CAMP061  CLG-ISP-926  Shankar Mandal  
2            CAMP062  CLG-ISP-504    Sachin Kumar  
3            CAMP112  CLG-ISP-509      Firoz Alam  
4            CAMP132  CLG-ISP-498           Vimal

In [9]:
StoreMasterRevisedxx = StoreMasterRevised[StoreMasterRevised['New_Amazing_EMP_ID']=='CAMP856']
StoreMasterRevisedxx

New_DT_Store_Code OLD_DT_Store_Code  LIS_StoreCode Old_Amazing_EMP_ID  \
504       DT03RRLFRLF               NSO  CLG-Store-914            CAMP856   

    New_Amazing_EMP_ID     LISUserId  EmployeeName  
504            CAMP856  CLG-ISP-1195  Tarannum Ara

#### Extracting Amazing Raw Data ( local path/ server path / google drive)

#### Route Data (Coverage Data)

In [10]:
coverage=pd.read_csv('1.Route_Data.csv',usecols = ['Check In Date','Check Out Date','Visit status','Route Day','Route Date','Store status','Merchandiser Code','Store Name','Store Code'])

C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_7428\2873854253.py:1: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  coverage=pd.read_csv('1.Route_Data.csv',usecols = ['Check In Date','Check Out Date','Visit status','Route Day','Route Date','Store status','Merchandiser Code','Store Name','Store Code'])


In [11]:
coverage['Merchandiser Code'] = (coverage['Merchandiser Code'].astype(str).str.replace('@zing.com', '', regex=False))

In [12]:
coverage['Merchandiser Code'] = (
    coverage['Merchandiser Code']
    .astype(str)
    .str.upper()
)
coverage.head()

Check In Date    Check Out Date Visit status Route Day  Route Date  \
0  31-12-2025 18:58  03-01-2026 10:52            Y  Saturday  03-01-2026   
1  31-12-2025 18:58  05-01-2026 17:46            Y  Saturday  03-01-2026   
2  31-12-2025 22:07  01-01-2026 22:07            Y    Friday  02-01-2026   
3  31-12-2025 22:07  01-01-2026 22:07            Y    Friday  02-01-2026   
4  31-12-2025 22:10  02-01-2026 22:05            Y  Thursday  01-01-2026   

         Store status Merchandiser Code            Store Name   Store Code  
0          Store open        IN0009M109           MEENA BAZAR   C000048173  
1          Store open        IN0009M109          THORAT BAZAR   C000048179  
2  Camera not allowed     IN0059APTGC09  DMA-L B NAGAR D-MART  DT04DMA5014  
3  Camera not allowed     IN0059APTGC09  DMA-L B NAGAR D-MART  DT04DMA5014  
4  Camera not allowed               X74     D MART - HADAPSAR  DT01DMA4151

#### Cleaning of camp id columm ( new amazing - consolidated)
Creating a Consolidated EMP id or Camp Id

In [13]:
temp = StoreMasterRevised[['Old_Amazing_EMP_ID','New_Amazing_EMP_ID']].drop_duplicates()

In [14]:
temp.head()

Old_Amazing_EMP_ID New_Amazing_EMP_ID
0               X157            CAMP059
1          IN0028M57            CAMP061
2         IN0028M154            CAMP062
3         IN0028M475            CAMP112
4         IN0028M326            CAMP132

In [15]:
coverage1 = pd.merge(coverage,temp,
                     left_on = 'Merchandiser Code',right_on='Old_Amazing_EMP_ID' , how = 'left')

In [16]:
# kunal

In [17]:
coverage2 = pd.merge(coverage1,StoreMasterRevised[['New_Amazing_EMP_ID']].drop_duplicates(),left_on='Merchandiser Code',right_on='New_Amazing_EMP_ID',
                     how='left',suffixes=('_old_map', '_new_map'))

In [18]:
coverage2['emp_flag'] = (coverage2[['New_Amazing_EMP_ID_old_map', 'New_Amazing_EMP_ID_new_map']].notna().any(axis=1).astype(int))

In [19]:
coverage2['Consolidated_NEW_EMP_ID'] = (coverage2['New_Amazing_EMP_ID_old_map'].combine_first(coverage2['New_Amazing_EMP_ID_new_map']))

In [20]:
coverage2.drop(columns=['Old_Amazing_EMP_ID','New_Amazing_EMP_ID_old_map', 'New_Amazing_EMP_ID_new_map'],inplace=True,errors='ignore')

In [21]:
coverage3= coverage2[(coverage2['emp_flag']==1)]

In [22]:
coverage3= coverage3.rename(columns = {'Check In Date':'CheckInDate','Check Out Date':'CheckOutDate','Visit status':'VisitStatus','Merchandiser Code':'MerchandiserCode',
                                      'Store Name':'StoreName','Store Code':'StoreCode'})

In [23]:
## NEED TO HAVE CAMP ID ALWYAS IN UPPER CASE IN MASTER ANN COVERAGE AND SO ON

coverage3['Consolidated_NEW_EMP_ID'] = (
    coverage3['Consolidated_NEW_EMP_ID']
    .astype(str)
    .str.upper()
)

In [24]:
coverage3= coverage3.drop(columns = ['MerchandiserCode','emp_flag'])

In [25]:
coverage3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8113 entries, 10 to 44244
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   CheckInDate              6742 non-null   object
 1   CheckOutDate             6446 non-null   object
 2   VisitStatus              8113 non-null   object
 3   Route Day                8113 non-null   object
 4   Route Date               8113 non-null   object
 5   Store status             8113 non-null   object
 6   StoreName                8113 non-null   object
 7   StoreCode                8113 non-null   object
 8   Consolidated_NEW_EMP_ID  8113 non-null   object
dtypes: object(9)
memory usage: 633.8+ KB


In [26]:
coverage3.head()

CheckInDate      CheckOutDate VisitStatus Route Day  Route Date  \
10  01-01-2026 05:32  01-01-2026 14:24           Y  Thursday  01-01-2026   
11  01-01-2026 05:47  01-01-2026 15:09           Y  Thursday  01-01-2026   
12  01-01-2026 05:53  01-01-2026 15:09           Y  Thursday  01-01-2026   
13  01-01-2026 05:53  01-01-2026 15:09           Y  Thursday  01-01-2026   
14  01-01-2026 05:57  01-01-2026 15:11           Y  Thursday  01-01-2026   

          Store status                   StoreName    StoreCode  \
10          Store open                     KOLKATA    DT03MET60   
11  Camera not allowed              Daman Guajarat  DT01DMA3951   
12  Camera not allowed  Kadipikonda Warangal DMart  DT04DMA5047   
13  Camera not allowed  Kadipikonda Warangal DMart  DT04DMA5047   
14  Camera not allowed                    Kompally  DT04DMA5021   

   Consolidated_NEW_EMP_ID  
10                 CAMP368  
11                 CAMP571  
12                 CAMP225  
13                 CAMP225  
14                 CAMP213

In [27]:
#time

In [28]:
coverage3['CheckInDate'] = pd.to_datetime(coverage3['CheckInDate'], errors='coerce')
coverage3['CheckOutDate'] = pd.to_datetime(coverage3['CheckOutDate'], errors='coerce')

coverage3['CheckIn_Time_24H'] = coverage3['CheckInDate'].dt.strftime('%H:%M')
coverage3['CheckOut_Time_24H'] = coverage3['CheckOutDate'].dt.strftime('%H:%M')


In [29]:
time_diff = (coverage3['CheckOutDate'] - coverage3['CheckInDate']).abs()

valid_mask = time_diff.notna()

coverage3['WorkingHours'] = np.nan
coverage3.loc[valid_mask, 'WorkingHours'] = (
    (time_diff[valid_mask].dt.total_seconds() // 3600).astype(int).astype(str).str.zfill(2)
    + ':' +
    ((time_diff[valid_mask].dt.total_seconds() % 3600) // 60).astype(int).astype(str).str.zfill(2)
)

C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_7428\3140017122.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['08:52' '09:22' '09:16' ... '00:05' '01:28' '00:14']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  coverage3.loc[valid_mask, 'WorkingHours'] = (


In [30]:
coverage3['Merchandising'] = np.where(
    (coverage3['VisitStatus'] == 'Y') &
    (coverage3['Store status'].isin(['Permission not granted', 'Store closed', '-'])),
    'N',
    coverage3['VisitStatus']
)

In [31]:
coverage3['Reason'] = np.where(
    (coverage3['VisitStatus'] == 'Y') & (coverage3['Merchandising'] == 'N'),
    coverage3['Store status'],
    np.nan
)

In [32]:
coverage3.head(2)

CheckInDate        CheckOutDate VisitStatus Route Day  Route Date  \
10 2026-01-01 05:32:00 2026-01-01 14:24:00           Y  Thursday  01-01-2026   
11 2026-01-01 05:47:00 2026-01-01 15:09:00           Y  Thursday  01-01-2026   

          Store status       StoreName    StoreCode Consolidated_NEW_EMP_ID  \
10          Store open         KOLKATA    DT03MET60                 CAMP368   
11  Camera not allowed  Daman Guajarat  DT01DMA3951                 CAMP571   

   CheckIn_Time_24H CheckOut_Time_24H WorkingHours Merchandising Reason  
10            05:32             14:24        08:52             Y    NaN  
11            05:47             15:09        09:22             Y    NaN

In [33]:
coverage4 = pd.merge(coverage3,StoreMasterRevised, left_on = ['StoreCode','Consolidated_NEW_EMP_ID'] , 
                     right_on= ['New_DT_Store_Code','New_Amazing_EMP_ID'],how = 'inner')

In [34]:
# coverage4.head()

In [35]:
coverage4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7664 entries, 0 to 7663
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   CheckInDate              6352 non-null   datetime64[ns]
 1   CheckOutDate             6058 non-null   datetime64[ns]
 2   VisitStatus              7664 non-null   object        
 3   Route Day                7664 non-null   object        
 4   Route Date               7664 non-null   object        
 5   Store status             7664 non-null   object        
 6   StoreName                7664 non-null   object        
 7   StoreCode                7664 non-null   object        
 8   Consolidated_NEW_EMP_ID  7664 non-null   object        
 9   CheckIn_Time_24H         6352 non-null   object        
 10  CheckOut_Time_24H        6058 non-null   object        
 11  WorkingHours             6058 non-null   object        
 12  Merchandising            7664 non-

In [44]:
coverage4 = coverage4[coverage4['VisitStatus']=='Y']

In [53]:
final_coverage_data= coverage4[['Consolidated_NEW_EMP_ID','LISUserId','EmployeeName','StoreCode','LIS_StoreCode','StoreName','Route Date','CheckIn_Time_24H','CheckOut_Time_24H','WorkingHours','VisitStatus','Merchandising','Reason']]

In [46]:
final_coverage_data.head(5)

Consolidated_NEW_EMP_ID     LISUserId                      EmployeeName  \
0                 CAMP368   CLG-ISP-910            SHIBSHANKAR CHATTERJEE   
1                 CAMP571   CLG-ISP-300  Ranjanben Arvindbhai Mahyavanshi   
2                 CAMP225   CLG-ISP-623                    KOTTEM MOUNIKA   
3                 CAMP225   CLG-ISP-623                    KOTTEM MOUNIKA   
4                 CAMP213  CLG-ISP-1261                   LAXMI PRAJAPATI   

     StoreCode  LIS_StoreCode                   StoreName  Route Date  \
0    DT03MET60   CLG-Store-33                     KOLKATA  01-01-2026   
1  DT01DMA3951  CLG-Store-298              Daman Guajarat  01-01-2026   
2  DT04DMA5047  CLG-Store-610  Kadipikonda Warangal DMart  01-01-2026   
3  DT04DMA5047  CLG-Store-610  Kadipikonda Warangal DMart  01-01-2026   
4  DT04DMA5021  CLG-Store-651                    Kompally  01-01-2026   

  CheckIn_Time_24H CheckOut_Time_24H WorkingHours Reason  
0            05:32             14:24        08:52    NaN  
1            05:47             15:09        09:22    NaN  
2            05:53             15:09        09:16    NaN  
3            05:53             15:09        09:16    NaN  
4            05:57             15:11        09:14    NaN

In [47]:
final_coverage_data.to_csv('final_coverage_data.csv')

In [49]:
final_coverage_data = final_coverage_data.drop_duplicates()

In [50]:
final_coverage_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4359 entries, 0 to 6325
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Consolidated_NEW_EMP_ID  4359 non-null   object
 1   LISUserId                4359 non-null   object
 2   EmployeeName             4359 non-null   object
 3   StoreCode                4359 non-null   object
 4   LIS_StoreCode            4359 non-null   object
 5   StoreName                4359 non-null   object
 6   Route Date               4359 non-null   object
 7   CheckIn_Time_24H         4359 non-null   object
 8   CheckOut_Time_24H        4359 non-null   object
 9   WorkingHours             4359 non-null   object
 10  Reason                   6 non-null      object
dtypes: object(11)
memory usage: 408.7+ KB
